### Model

In [1]:
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_csv(r'c:\Users\Администратор\Documents\study\CSME 830\house_price_project\data\raw\cleaned.csv')
df = pd.read_csv(r'c:\Users\Администратор\Documents\study\CSME 830\house_price_project\models\re_cleaned.csv')
df.columns

Index(['Unnamed: 0', 'status', 'bed', 'bath', 'acre_lot', 'city', 'state',
       'zip_code', 'house_size', 'price', 'has_prev_sale',
       'years_since_prev_sale', 'house_size_log', 'acre_lot_log', 'price_log',
       'zip_code_encoded', 'city_encoded', 'state_encoded', 'status_encoded',
       'zip_freq', 'city_freq', 'bed_bath_interactions', 'size_per_bed',
       'bath_per_bed', 'size_bath_interaction'],
      dtype='object')

In [13]:
df = df.drop(df[df['price'] == 0].index, inplace=True)

In [3]:
features = [
    "price",
    # "bed",
    # "bath",
    "bed_bath_interactions",
    "house_size",
    # "house_size_log",
    "acre_lot_log",
    "zip_code_encoded",
    "city_encoded",
    "state_encoded",
    "zip_freq",
    "city_freq",
    "size_per_bed",
    "bath_per_bed",
    "size_bath_interaction"
]

df_model = df[features]

### CatBooostRegressor (best one)

In [4]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

# Target
X = df_model.drop(columns=['price']) 
y = np.log1p(df_model["price"])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("Split is complete.")

Split is complete.


In [5]:
# Model
model = CatBoostRegressor(
    iterations=3000,
    depth=10,
    learning_rate=0.03,
    l2_leaf_reg=5,
    loss_function="RMSE",
    bagging_temperature=0.5,
    random_seed=42,
    verbose=200
)


# Train
model.fit(X_train, y_train)
print('model training is complete.')
# Score
print("Train R2:", model.score(X_train, y_train))
print("Test R2:", model.score(X_test, y_test))


0:	learn: 0.7998909	total: 177ms	remaining: 8m 51s
200:	learn: 0.3958896	total: 6.37s	remaining: 1m 28s
400:	learn: 0.3783840	total: 12.7s	remaining: 1m 22s
600:	learn: 0.3648621	total: 18.9s	remaining: 1m 15s
800:	learn: 0.3543011	total: 25.1s	remaining: 1m 8s
1000:	learn: 0.3449474	total: 31.6s	remaining: 1m 3s
1200:	learn: 0.3376642	total: 37.7s	remaining: 56.5s
1400:	learn: 0.3314071	total: 43.9s	remaining: 50.1s
1600:	learn: 0.3257668	total: 49.9s	remaining: 43.6s
1800:	learn: 0.3204830	total: 56.2s	remaining: 37.4s
2000:	learn: 0.3154840	total: 1m 2s	remaining: 31.1s
2200:	learn: 0.3109099	total: 1m 8s	remaining: 24.8s
2400:	learn: 0.3060317	total: 1m 14s	remaining: 18.6s
2600:	learn: 0.3015414	total: 1m 21s	remaining: 12.5s
2800:	learn: 0.2974198	total: 1m 27s	remaining: 6.24s
2999:	learn: 0.2934724	total: 1m 33s	remaining: 0us
model training is complete.
Train R2: 0.870414414146086
Test R2: 0.7984398846738608


First attempt with house_size_log and bath, bed: 

model training is complete.
- Train R2: 0.8629312966428447
- Test R2: 0.7915728470631084

In [6]:
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')

print(scores)
print("Mean CV R²:", scores.mean())


0:	learn: 0.7998909	total: 85.8ms	remaining: 4m 17s
200:	learn: 0.3963432	total: 6.27s	remaining: 1m 27s
400:	learn: 0.3792138	total: 12.1s	remaining: 1m 18s
600:	learn: 0.3657718	total: 17.8s	remaining: 1m 11s
800:	learn: 0.3554354	total: 23.7s	remaining: 1m 5s
1000:	learn: 0.3461657	total: 29.6s	remaining: 59.1s
1200:	learn: 0.3378009	total: 35.8s	remaining: 53.6s
1400:	learn: 0.3311540	total: 41.8s	remaining: 47.7s
1600:	learn: 0.3254623	total: 47.9s	remaining: 41.8s
1800:	learn: 0.3203000	total: 53.9s	remaining: 35.9s
2000:	learn: 0.3153859	total: 59.8s	remaining: 29.9s
2200:	learn: 0.3105122	total: 1m 5s	remaining: 23.9s
2400:	learn: 0.3059957	total: 1m 11s	remaining: 17.9s
2600:	learn: 0.3016270	total: 1m 17s	remaining: 11.9s
2800:	learn: 0.2974239	total: 1m 23s	remaining: 5.95s
2999:	learn: 0.2935052	total: 1m 29s	remaining: 0us
0:	learn: 0.7935581	total: 40.4ms	remaining: 2m 1s
200:	learn: 0.3893726	total: 6.28s	remaining: 1m 27s
400:	learn: 0.3712926	total: 12.1s	remaining: 1m

In [9]:
import joblib

# Save ONLY the model
joblib.dump(model, 'catboost_model.pkl')
#catboost_model.pkl

print("Model saved successfully!")

Model saved successfully!


In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # back-transform
y_real = np.expm1(y_test)

rmse = np.sqrt(mean_squared_error(y_real, y_pred))
mae = mean_absolute_error(y_real, y_pred)
print("RMSE ($):", rmse)
print("MAE ($):", mae)


RMSE ($): 326709.48477718217
MAE ($): 109352.39203186255


In [ ]:
# function to make the raw data ready for a model


In [15]:
model.get_feature_importance(prettified=True)


,Feature Id,Importances
0,zip_code_encoded,28.305273
1,zip_freq,15.240761
2,size_bath_interaction,11.253086
3,state_encoded,8.663385
4,city_encoded,8.582574
5,house_size,8.271100
6,acre_lot_log,6.544499
7,city_freq,4.991587
8,size_per_bed,3.649327
9,bath_per_bed,2.568636


In [ ]:
# y_pred = model.predict(X_test)
# y_pred_real = np.expm1(y_pred)

# Later when I want to predict the price i need to convert it back to normal from log

### LinearRegression

In [17]:
from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# train
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)


In [18]:
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# 0. R-Squared (The "Accuracy" equivalent)
r2 = r2_score(y_test, predictions)
print(f"R-squared Score: {r2:.2f} (or {r2*100:.1f}%)")

# 1. Reverse the log transformation
y_test_dollars = np.expm1(y_test)       # The real prices
pred_dollars = np.expm1(predictions)    # Your model's predicted prices

# 2. Check the Real Error
mae = mean_absolute_error(y_test_dollars, pred_dollars)
rmse = np.sqrt(mean_squared_error(y_test_dollars, pred_dollars))

print(f"Real Dollar Error (MAE): ${mae:,.0f}")
print(f"Real RMSE: ${rmse:,.0f}")

R-squared Score: 0.58 (or 58.5%)
Real Dollar Error (MAE): $170,404
Real RMSE: $472,233


### RandomForest (second best)

In [ ]:
# Implementation of RandomForest
from sklearn.ensemble import RandomForestRegressor

# 1.Initialization
rf_model = RandomForestRegressor()

# 2. Fit on the model
rf_model.fit(X_train, y_train)

# 3. Predicitons
log_predictions = rf_model.predict(X_test)

# 4. Check the scores
final_predictions = np.expm1x(log_predictions)
real_y_test = np.expm1x(y_test)



NameError: name 'r2_score' is not defined

In [15]:
from sklearn.metrics import r2_score, mean_squared_error

print(f"Random Forest R2: {r2_score(real_y_test, final_predictions):.2f}")

print(f"Random Forest R2: {r2_score(real_y_test, final_predictions):.2f}")

Random Forest R2: 0.77
Random Forest R2: 0.77


In [16]:

print(f"R2 Score: {r2_score(real_y_test, final_predictions):.4f}")
print(f"MAE: ${mean_absolute_error(real_y_test, final_predictions):,.0f}")

R2 Score: 0.7750


NameError: name 'mean_absolute_error' is not defined